## Import Libraries and Declare API Key

In [1]:
import requests
import pandas as pd
import numpy as np
import signal
import matplotlib.pyplot as plt
import talib
import time

In [2]:
API_KEY = {'X-API-key': 'RPCJTKLW'}

## Basic Buy and Sell Functions

In [3]:
shutdown = False

class ApiException(Exception):
    pass

def signal_handler(signum, frame):
    global shutdown
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown = True

def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Authorization error. Please check API key.')

def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Authorization error. Please check API key.')

def buy(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'BUY'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'BUY', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def sell(session, ticker, type, q, price = None):
    if type == 'MARKET':
        resp = session.post('http://localhost:9999/v1/orders', params = {'ticker': ticker, 'type': 'MARKET', 'quantity': q, 'action': 'SELL'})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
    else:
        resp = session.post('http://localhost:9999/v1/orders',
                            params={'ticker': ticker, 'type': 'LIMIT', 'quantity': q, 'action': 'SELL', 'price': price})
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')

def buy_mult(session, ticker, type, mult, price = None):
    for i in range(mult):
        buy(session, ticker, type, 10000, price)
        
def sell_mult(session, ticker, type, mult, price = None):
    for i in range(mult):
        sell(session, ticker, type, 10000, price)

def cancel(session, order_id):
    order_id = order_id # assuming the order to cancel has ID 100
    resp = s.delete('http://localhost:9999/v1/orders/{}'.format(order_id))
    if resp.ok:
        status = resp.json()
        success = status['success']
        print('The order was successfully cancelled?', success)
        

In [4]:
# with requests.Session() as s:
#     df_ngn = None
#     df_whel = None
#     df_gear = None
#     df_rsm = None

#     s.headers.update(API_KEY)
#     resp1 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
#     resp2 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'WHEL'})
#     resp3 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'GEAR'})
#     resp4 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'RSM1000'})
#     if resp1.ok and resp2.ok and resp3.ok and resp4.ok:
#         book1 = resp1.json()
#         book2 = resp2.json()
#         book3 = resp3.json()
#         book4 = resp4.json()
#         df_ngn = pd.DataFrame(book1)
#         df_whel = pd.DataFrame(book2)
#         df_gear = pd.DataFrame(book3)
#         df_rsm = pd.DataFrame(book4)
#     else:
#         print('request denied.')

In [5]:
# real = talib.RSI(df_gear['close'].values, timeperiod=10)
# print(real)

## Algorithm Execution Section

In [ ]:
signal.signal(signal.SIGINT, signal_handler)
price = 10000

while True:
    with requests.Session() as s:
        df_ngn = None
        df_whel = None
        df_gear = None
        df_rsm = None
        
        s.headers.update(API_KEY)
        resp1 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'NGN'})
        resp2 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'WHEL'})
        resp3 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'GEAR'})
        resp4 = s.get('http://localhost:9999/v1/securities/history', params = {'ticker' : 'RSM1000'})
        
        resp_gear_pos = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'GEAR'})
        gear_pos = resp_gear_pos.json()[0]['position']
        if resp1.ok and resp2.ok and resp3.ok and resp4.ok:
            book1 = resp1.json()
            book2 = resp2.json()
            book3 = resp3.json()
            book4 = resp4.json()
            df_ngn = pd.DataFrame(book1)
            df_whel = pd.DataFrame(book2)
            df_gear = pd.DataFrame(book3)
            df_rsm = pd.DataFrame(book4)
        else:
            print('request denied.')
            
            
        _, _, macdhist_gear = talib.MACD(df_gear['close'].values, 25 , 60, 25)
#         _, _, macdhist_whel = talib.MACD(df_whel['close'].values, 5 , 12, 5)
#         _, _, macdhist_ngn = talib.MACD(df_ngn['close'].values, 5 , 12, 5)
        
        if macdhist_gear.size > 85:
            if macdhist_gear[83] > 0 and -2 < macdhist_gear[84] < 0.001:
                print('macd sell: ', macdhist_gear[83], macdhist_gear[84])
                sell(s, 'GEAR','MARKET', price)
            if macdhist_gear[83] < 0 and -0.009 < macdhist_gear[84] < 2:
                print('macd buy: ', macdhist_gear[83], macdhist_gear[84])
                buy(s, 'GEAR','MARKET', price)

        real_gr = talib.RSI(df_gear['close'].values, timeperiod=5)
    
        if real_gr.size > 5:
            if real_gr[5] > 40 and real_gr[6] < 40 and gear_pos > 0:
                print('rsi sell: ', real_gr[5])
                sell_mult(s, 'GEAR','MARKET', int(gear_pos/10000))
                
            if real_gr[5] < 40 and real_gr[6] > 40 and gear_pos < 0:
                print('rsi buy: ',real_gr[5])
                buy_mult(s, 'GEAR','MARKET', int(gear_pos/10000))
                
    time.sleep(1)

rsi sell:  41.46341463414638
The market buy order was submitted and has ID 1259
The market buy order was submitted and has ID 1260
macd sell:  0.007111008494865262 -0.005511050908927867
The market buy order was submitted and has ID 1461
rsi buy:  27.083333333333304
macd buy:  -0.0038698197253779765 0.0030194377325071797
The market buy order was submitted and has ID 1558
macd buy:  -0.0022344921244553118 -0.008646316874836024
The market buy order was submitted and has ID 1599
macd sell:  0.0017548664447804285 -0.0006622533311454737
The market buy order was submitted and has ID 1608
macd buy:  -0.010958933848412039 0.007260670430906002
The market buy order was submitted and has ID 1769
rsi sell:  52.38095238095269
The market buy order was submitted and has ID 1810
macd sell:  0.005422765491752551 -0.005261965681804148
The market buy order was submitted and has ID 1851
rsi buy:  39.39393939393952
rsi buy:  33.33333333333333
rsi buy:  25.925925925925732
rsi buy:  34.54545454545473
rsi buy:

In [ ]:
# sell(s, 'GEAR','MARKET', 1000)

# with requests.Session() as s:
#     s.headers.update(API_KEY)
#     resp_gear_pos = s.get('http://localhost:9999/v1/securities', params = {'ticker' : 'GEAR'})
#     gear_pos = resp_gear_pos.json()[0]['position']
#     print(gear_pos)
    